# Bibliotecas

In [1]:
!pip install pyngrok
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 4.0 MB/s eta 0:00:00


In [2]:
import os
import glob
import shutil
from pyngrok import ngrok
from getpass import getpass
from google.colab import drive

# Arquivos do projeto

In [3]:
# Área de trabalho do projeto
drive.mount('/content/drive')
workspace_path = '/content/drive/MyDrive/Workspace'

Mounted at /content/drive


In [4]:
# Caminho para os arquivos dos dados
data_path = f'{workspace_path}/fiap/pos-tech/data-analytics/fase-4/dados'

In [5]:
# Pasta temporária
temp_path = 'tmp'
os.makedirs(temp_path, exist_ok=True)

In [6]:
# Copiando os arquivos do backup do banco de dados
for part_file in glob.glob(f'{data_path}/anp_dump.tar.gz.part-*'):
    shutil.copy(part_file, temp_path)

In [7]:
# Remontando o arquivo de backup no formato .tar.gz
!cat tmp/anp_dump.tar.gz.part-* > tmp/anp_dump.tar.gz

In [8]:
# Descompactando o arquivo de backup do banco de dados
!tar -xzvf tmp/anp_dump.tar.gz

tmp/anp_dump.backup


# PostgreSQL

In [9]:
# Instalação
!apt-get install -y postgresql

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert
  sysstat
0 upgraded, 13 newly installed, 0 to remove and 49 not upgraded.
Need to get 18.4 MB of archives.
After this operation, 51.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd6

In [10]:
# Iniciando o serviço
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [11]:
# Configurando o usuário postgres
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"

ALTER ROLE


In [12]:
# Criando o banco de dados
!sudo -u postgres psql -c "CREATE DATABASE anp;"

CREATE DATABASE


In [13]:
# Restaurando os dados para o PostgreSQL
!PGPASSWORD=postgres pg_restore -U postgres -d anp -h localhost -v tmp/anp_dump.backup

pg_restore: connecting to database for restore
pg_restore: creating TABLE "public.preco_combustivel"
pg_restore: processing data for table "public.preco_combustivel"


# Streamlit

In [14]:
with open('app.py', 'w') as f:
    f.write("""
import psycopg2
import streamlit as st
import pandas as pd
import plotly.express as px

# Conexão com o PostgreSQL
conn = psycopg2.connect(
    host='localhost',
    port='5432',
    database='anp',
    user='postgres',
    password='postgres'
)

# Carregando os dados do PostgreSQL em um dataframe Pandas
df = pd.io.sql.read_sql_query('''
  SELECT  estado_sigla      AS estado
          , produto
          , valor_de_venda  AS valor_venda
          , data_da_coleta  AS data_coleta
  FROM    public.preco_combustivel
''', conn)

# Ajustando a coluna data_da_coleta
df['data_coleta'] = pd.to_datetime(df['data_coleta'], format='%Y-%m-%d')

# Ano e mês
df['mes_ano'] = df['data_coleta'].dt.to_period('M').astype(str)

# Estrutura da página
st.title('Preço de combustíveis (2018-2022)')

# Filtros
st.sidebar.header('Filtros')
selected_state = st.sidebar.selectbox('Estado', df['estado'].unique())
selected_product = st.sidebar.selectbox('Produto', df['produto'].unique())

filtered_data = df[(df['estado'] == selected_state) &
                   (df['produto'] == selected_product)]

# Evolução dos preços
st.subheader('Evolução dos preços')
df_avg_price = filtered_data.groupby('mes_ano', as_index=False)['valor_venda'] \
                            .mean()
fig_line = px.line(df_avg_price, x='mes_ano', y='valor_venda',
                   title='Evolução dos preços')
st.plotly_chart(fig_line)

# Resumo
st.subheader('Resumo')
st.write(filtered_data[['valor_venda']].describe())
""")

In [15]:
!streamlit run app.py &> /dev/null&

# ngrok

In [ ]:
ngrok_token = getpass("ngrok token: ")
!ngrok config add-authtoken $ngrok_token

In [ ]:
public_url = ngrok.connect(addr="8501", proto="http")
print("Streamlit app URL:", public_url)